In [1]:
!pip install "https://github.com/megagonlabs/ginza/releases/download/latest/ginza-latest.tar.gz"

     |████████████████████████████████| 54.6MB 112kB/s 
     |████████████████████████████████| 71kB 4.6MB/s 
     |████████████████████████████████| 70.7MB 119kB/s 
     |████████████████████████████████| 481kB 9.7MB/s 
  Created wheel for ginza: filename=ginza-2.2.1-cp37-none-any.whl size=11779 sha256=8eefe2329937115f47421b49a0aaa8a88e8647ec56e430187b079f14d46bc4b2
  Stored in directory: /root/.cache/pip/wheels/6f/97/43/f9c8c315bcd598f595e96aae207b120a3b84564f03ddd8d6a7
  Created wheel for ja-ginza: filename=ja_ginza-2.2.0-cp37-none-any.whl size=54672046 sha256=a84dd2e537dfa44544c8c317bf9d79b91d7f8ed2d5a57477fa257b1e2a731b54
  Stored in directory: /root/.cache/pip/wheels/21/63/1e/21ca7586a7abd427e0980f39a66549ba5d2543528b97253952
  Created wheel for SudachiPy: filename=SudachiPy-0.5.2-cp37-cp37m-linux_x86_64.whl size=870172 sha256=0b58be37134f6dd27990e6008fe6c67308eefb1e023bf1c214a0cb8dfc80ac1d
  Stored in directory: /root/.cache/pip/wheels/f1/cc/90/8c68725c622127bb3e5dc46756a8c03bd5

In [2]:
import pkg_resources, imp
imp.reload(pkg_resources)

<module 'pkg_resources' from '/usr/local/lib/python3.7/dist-packages/pkg_resources/__init__.py'>

https://www.nogawanogawa.com/entry/ginza  
https://www.mojirca.com/2019/10/colab-load-ginza.html  
ランタイムの再起動はやはり必要そう  

In [1]:
import spacy
from spacy import displacy
# GiNZAモデルの読み込み
nlp = spacy.load('ja_ginza')

In [2]:
# googleドライブをマウント
from google.colab import drive 
drive.mount('/content/drive')
# 表示されるリンクをクリックして、アクセスを許可して、最後に表示される文字列を以下の入力欄に入れる

Mounted at /content/drive


In [3]:
# 作業フォルダに移動
%cd '/content/drive/My Drive/work/'

/content/drive/My Drive/work


In [13]:
import json

input_file = "output_0001.json"
path_w = 'train_0001.txt'

count_length = 0

with open(path_w, mode='w') as f_out:
  with open(input_file) as f_in:
    json_text = json.load(f_in)
    for v_result in json_text.values():
      for v_alternatives in v_result:
        for val_list in v_alternatives.values():
          if isinstance(val_list, list):
            for val in val_list:
              count_length += len(val.get('transcript'))
              f_out.write(val.get('transcript'))
              f_out.write('。\n')

print('文字数')
print(count_length)

4171


In [14]:
filepath = "train_0001.txt"
with open(filepath) as f:
    s = f.read()

# 文章全体を解析
doc = nlp(s) 

In [15]:
#抽出したエンティティの種類に対して色を指定する
colors = {"COUNTRY":"#00cc00", "CITY":"#00cc00", "GPE_OTHER":"#00cc00","OCCASION_OTHER":"#00cc00",
          "LOCATION":"#00cc00", "LOCATION_OTHER":"#00cc00","DOMESTIC_REGION":"#00cc00","PROVINCE":"#00cc00",
          "STATION":"#00cc00", "CONTINENTAL_REGION":"#00cc00","THEATER":"#00cc00",

          "TIME":"#adff2f","DATE":"#adff2f","DAY_OF_WEEK":"#adff2f",
          "PERIOD_YEAR":"#adff2f", "PERIOD_MONTH":"#adff2f", "PERIOD_DAY":"#adff2f",

          "FLORA":"#adff99","FLORA_PART":"#adff99",
          "DISH":"#ffeb99","FOOD_OTHER":"#ffeb99",

          "AGE":"#3385ff","N_PERSON":"#3385ff","N_EVENT":"#3385ff","N_LOCATION_OTHER":"#3385ff","RANK":"#3385ff",
          "N_PRODUCT":"#3385ff","":"#3385ff","":"#3385ff","":"#3385ff","MEASUREMENT_OTHER":"#3385ff","PERCENT":"#3385ff",
          "N_ORGANIZATION":"#3385ff", "ORDINAL_NUMBER":"#3385ff", "N_FACILITY":"#3385ff","SPEED":"#3385ff",
          "PHONE_NUMBER":"#3385ff",

          "MONEY":"#ffff00",

          "COMPANY":"#99c2ff", "SCHOOL":"#99c2ff", "INTERNATIONAL_ORGANIZATION":"#99c2ff",
          "GOE_OTHER":"#99c2ff", "SHOW_ORGANIZATION":"#99c2ff","CORPORATION_OTHER":"#99c2ff",

          "CLOTHING":"#ff66a3",
          "PRODUCT_OTHER":"#ff66a3",

          "PERSON":"#c266ff",
          "POSITION_VOCATION":"#ebccff",

          "MUSIC":"#ff7f50", "MOVIE":"#ff7f50", "GAME":"#ff7f50", "SPORT":"#ff7f50", "BOOK":"#ff7f50", 
          "BROADCAST_PROGRAM":"#ff7f50", 

          "ANIMAL_DISEASE":"#cd5c5c"
          }

options = {"colors": colors}
displacy.render(doc, style="ent", options=options, jupyter=True)

In [19]:
result_text = ''
# 文単位でループ
for sent in doc.sents:
  # 形態素単位でループ
  for token in sent:
    hasText = True
    # 固有表現の取得
    for ent in sent.ents:
      # 置換しない固有表現
      if ent.label_ in ['LOC', 'DATE', 'TIME', 'PERCENT']:
        continue
      # 固有表現のテキストが一致した場合置換
      if token.text == ent.text:
        hasText = False
        # ent.label_ の意味合い
        # 組織名 (ORGANIZATION)、人名 (PERSON)、地名 (LOCATION)、日付表現 (DATE)、時間表現 (TIME)、金額表現 (MONEY)、割合表現 (PERCENT)
        result_text = result_text + ent.label_
        # 一度一致したら次の形態素の解析に移る
        break
    if hasText:
      # 固有表現がないのでそのまま保持
      result_text = result_text + token.text
  # 文章単位で改行
  result_text = result_text + '\n'

In [20]:
result_text

'神戸市立中央体育館です\n2021年西日本では初めての試合ということになります失礼します\n野菜の工業ということになります\n今日は西日本全国各地で雨が降っています\n春の嵐となっていますが点いたり時に冷たい雨が降ったりまさに三寒四温春の訪れを感じさせるような雨でもどうなっています。\n青コーナーから入場してきました\n明美智子東京世田谷大蔵ジムジムで戦って2勝4敗\nこれが7000目ということになりますデビューしてからなかなか勝てずに3連敗からの2連勝もあり昨年の10月に敗れて再びこれが作戦ということになります\n強い気持ちを持ってどんどん前に出る阿比留通子のボクシング今日も見られるんでしょうか。\n階段の前で一つ精神統一リングを見ました。\n階段を踏みしめるように上がって今リング院です\n阿比留通子青コーナーから入場しました。\n女子フライ級4回戦で行われる阿比留通子対浜田麻里奈の一閃阿比留通子三十歳。\nもう破れた後に練習を回ってピンチの中でもしっかり気持ちで負けないメンタルを使ってきましたという風に全然語っていました\nそして赤コーナーから入場してきました\n浜田麻里な兵庫県神戸市出身地元の試合ということもあって大きな拍手に包まれています\n神戸市立中央体育館です\n日本女子フライ級5位に位置しているランクインしている浜田麻里奈入場してきました。\n御堂筋。\n階段を上っていきます\nこれが4000目\nこれまでデビューしてから負けを知りません浜田麻里奈今リング院です\n地元の観客からは大きな拍手が送られていますかまだマイナーこれが4点目\n地元での試合ということになります\n観客に向かってメールをして試合に臨みます。\n阿比留通子30回東京世田谷大蔵事務職ですが凛となる試合となります。\n女子フライ級4回戦で行われます\n阿比留通子対浜田麻里奈の一閃2人にとってさらに上のランクに上がるためには勝たなければいけない試合ということになります。\n気合いは十分浴びるPERSON。\n女子プロ野球4回戦。\n阿比留通子たい濱田満里奈の一戦です\nこの試合の解説は真正ジム伊賀かおりさんにお越しいただいています\nどうぞよろしくお願いしますが売っていたのは阿比留通子ですが3個の試合女子フライ級4回戦で行われますがまずはどういったところがポイントになってきそうですかそ

In [21]:
path_w = 'train_0001_ent.txt'
with open(path_w, mode='w') as f_out:
  f_out.write(result_text)